In [3]:
from flask import Flask, request

from flask_cors import CORS, cross_origin
from flask_restful import Resource, Api
from json import dumps
from flask_jsonpify import jsonify
import json
import pymongo
import urllib
from collections import Counter
import requests
import time
import cryptography
import jwt
from flask import (Flask, make_response, render_template, redirect, request,url_for)

LIFE_SPAN = 180

SyntaxError: invalid syntax (jwk.py, line 60)

In [ ]:
def checkCredentails(request):
    username=request["username"]
    password=request["password"]
    
    if not authenticate_user_credentials(username, password):
        return json.dumps({
          "error": "access_denied"
        }), 401
  
    access_token = generate_access_token(username)
    db=client.credentials
    tokenDetails={}
    tokenDetails["userName"]=username
    tokenDetails["access_token"]=access_token    
    db.tokenKeys.insert_one(tokenDetails)
    
    return json.dumps({ 
    "access_token": access_token,
    "token_type": "JWT",
    "expires_in": LIFE_SPAN,
    'userName':username
    })


def generate_access_token(username):
    payload = {
    "iss": ISSUER,
    "exp": time.time() + LIFE_SPAN,
    "time":time.time(),
    "userName": username 
    }

    access_token = jwt.encode(payload, private_key, algorithm = 'RS256')

    return access_token.decode()

def authenticate_user_credentials(username, password):
    db=client.credentials
    form=db.userDetails.find_one({'userName': username,'password':password})
    
    if(form):
        return True
    return False

In [ ]:
ISSUER = 'SurveyShrike OAuth Server'


client = pymongo.MongoClient("mongodb+srv://hemanth:"+ urllib.quote("Hemanth@123")+"@cluster0-59obx.mongodb.net/test?retryWrites=true&w=majority")

with open('public.pem', 'rb') as f:
    public_key = f.read()
    
with open('private.pem', 'rb') as f:
    private_key = f.read()

In [2]:
app = Flask(__name__)
api = Api(app)
CORS(app)

@app.route('/login',methods=['POST'])
def request_token():

    params=json.loads(str(request.data))
    print(params)
    response = checkCredentails(params)

    return response

@app.route('/register',methods=['POST'])
def register_user():
    params=request.json
    print(params)
    db=client.credentials
    db.userDetails.find_one({'userName': params["username"]})
    return json.dumps({
          "error": "UserName Already Exists"
        }), 401
    db.userDetails.insert_one({'userName': params["username"],'password':params["password"]})
    response = checkCredentails(params)
    return response


@app.route('/logout',methods=['POST'])
def delete_token():
    access_token=request.headers["Authorization"].encode()
    try:
        decoded_token = jwt.decode(access_token, public_key,
                                   issuer = ISSUER,
                                   algorithm = 'RS256')
    except (jwt.exceptions.InvalidTokenError,
          jwt.exceptions.InvalidSignatureError,
          jwt.exceptions.InvalidIssuerError,
          jwt.exceptions.ExpiredSignatureError):
        return json.dumps({
              "error": "access_denied"
            }), 401
    
    db=client.credentials
    db.tokenKeys.delete_one({"access_token":access_token})
    return json.dumps({
              "message": "Successfully Logged out"
        })
    
    return response


@app.route('/authorize', methods = ['POST'])
def verify_access_token():
    try:
        decoded_token = jwt.decode(request.headers["Authorization"].encode(), public_key,
                                   issuer = ISSUER,
                                   algorithm = 'RS256')
    except (jwt.exceptions.InvalidTokenError,
          jwt.exceptions.InvalidSignatureError,
          jwt.exceptions.InvalidIssuerError,
          jwt.exceptions.ExpiredSignatureError):
        return json.dumps({
              "error": "access_denied"
            }), 401

    return json.dumps({
              "message": "Valid User"
            })

if __name__ == '__main__':
    app.run(port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2019 01:50:41] "OPTIONS /login HTTP/1.1" 200 -
[2019-10-20 01:50:41,140] ERROR in app: Exception on /login [POST]
Traceback (most recent call last):
  File "/Users/hemanth/anaconda2/lib/python2.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/hemanth/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/hemanth/anaconda2/lib/python2.7/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/hemanth/anaconda2/lib/python2.7/site-packages/flask_restful/__init__.py", line 269, in error_router
    return original_handler(e)
  File "/Users/hemanth/anaconda2/lib/python2.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value

{u'username': u'User1', u'password': u'User1'}
